## Interprete per espressioni aritmetiche

Si consideri la seguente sintassi delle espressioni aritmetiche, estesa con le variabili (`Ide`) e un costrutto sommatoria (`Sumall`)

    Exp ::= N | Ide | Exp op Exp | sumall ( Ide ; n ; n ){Exp}
    op  ::= + | - | * | / 
    N   ::= 0 | 1 | 2 | ...
    Ide ::= x | y | z | ...

La sua sintassi astratta è definita come:

In [1]:
(*Sintassi astratta*)
type op = Add | Sub | Mul | Div ;;
type exp = 
    | Val of int
    | Ide of string
    | Op of op*exp*exp
    | Sumall of string*int*int*exp;; 

type op = Add | Sub | Mul | Div


type exp =
    Val of int
  | Ide of string
  | Op of op * exp * exp
  | Sumall of string * int * int * exp


### Es 1:

Nel linguaggio esteso come da esercizio precedente, si implementi una funzione `fv e` che prende una espressione `e` e restituisce **l'insieme delle sue variabili libere**, definito come segue.

    fv(n) = {}
    fv(Ide) = {Ide}
    fv(E1 op E2) = fv(E1) U fv(E2)
    fv(sumall (Ide; n; n;) Exp) = fv(Exp) \ Ide

Per rappresentare un insieme di variabili, possiamo usare una lista senza duplicati. Torvate qui sotto due funzioni già implementate per manipolare questi "insiemi", ovvero l'unione fra due insiemi e la rimozione di un elemento da un insieme.

In [ ]:
let union l1 l2 =
    let rec remove_dup l = match l with
        | [] -> []
        | x :: xs -> x :: remove_dup (List.filter (fun y -> y <> x) xs)
    in remove_dup (l1 @ l2);;

let remove l el = List.filter (fun y -> y <> el) l;;

### Es 2:
Si implementi una funzione `subst (e:exp) (x:ide) (v:exp)` dove `e` è un'espressione, `x` è un identificatore e `v` è un'espressione che deve descrivere un valore (`Val n`). La funzione calcola l'espressione $e\{x:=v\}$ in cui tutte le occorrenze "libere" di `x` in `e` sono sostituite con `v`. 

**ATTENZIONE:** Il costrutto `sumall (x;n1;n2) {e}` lega le occorrenza di `x` in `e`. La sostituzione deve operare solo sulle occorrenze libere della variabile. 

**TEST:** applicare la funzione `subst` in modo da calcolare $((x+y)+(\mbox{sumall }x;1;10) \, x) \{x:=5\}$ che deve restituire $((5+y)+(\mbox{sumall }x;1;10)\, x)$.

In [ ]:
let e1 = Op (Add, (Op (Add, Ide "x", Ide "y")), (Sumall ("x",1,10,Ide "x"))) in
subst e1 "x" (Val 5);;

### Es 3:

Si modifichi il codice (riportato qui sotto) dell'interprete big step delle espressioni in modo tenere conto della semantica estesa con il costrutto `sumall`, definita come segue. Per l'operazione di sostituzione $e \{x := v\}$ utilizzare la funzione `subst` definita nell'esercizio precedente.

$$
n \rightarrow_{bs} n
\qquad
\frac{E_1 \rightarrow_{bs} n_1 \quad E_2 \rightarrow_{bs} n_2 \quad n_1 \mbox{op } n_2 = n}{E_1 \, op \, E_1 \rightarrow_{bs} n}
$$

$$
\frac{e\{x:=n\} \rightarrow_{bs} n'}
{\mbox{sumall } ( x ; n ; n ) \, e \rightarrow_{bs} n'}
$$

$$
\frac{n_1<n_2 \quad e\{x:=n_1\} \rightarrow_{bs} n' \quad \mbox{sumall } ( x ; n_1+1 ; n_2 ) \, e \rightarrow_{bs} n'' \quad n = n' + n''}
{\mbox{sumall } ( x ; n_1 ; n_2 ) \, e \rightarrow_{bs} n}
$$

**NOTA1:** non sono previste regole per gli identificatori (sono gestiti dalla sostituzione).

**NOTA2:** la semantica non prevede transizioni in caso di identificatori liberi (non legati da `sumall`) o indici numerici usati nel costrutto `sumall` non ben definiti

In [ ]:
(* interprete big-step *)
let rec eval e =
    match e with
    | Val n -> Val n
    | Op (op,e1,e2) -> 
        (* chiamate ricorsive che calcolano le derivazioni per e1 ed e2 *)
        (match (eval e1, eval e2) with
        | (Val n1, Val n2) -> (match op with   (* calcola n1 op n2 *)
                               | Add -> Val (n1+n2)
                               | Sub -> Val (n1-n2)
                               | Mul -> Val (n1*n2)
                               | Div -> Val (n1/n2)
                               )
        (* caso (inutile) aggiunto solo per rendere esaustivo il pattern matching *)
        | _ -> failwith "Errore impossibile che si verifichi" )
    | ??? ;;

**TEST:** calcolare il risultato delle seguenti espressioni:

* sumall (x; 1; 10) x
* 100 + sumall (x;1;10) (sumall (y;1;10) x*y)

Risultati attesi: 55 e 3125

In [ ]:
let e2 = Sumall ("x",1,10,Ide "x");;
let e3 = Op (Add, Val 100, 
                 (Sumall ("x",1,10,(Sumall ("y",1,10, Op ( Mul, Ide "x", Ide "y")))))) ;;
                 
eval e2;;
eval e3;;